In [1]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes

# Generate group's master key pair
group_private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)
group_public_key = group_private_key.public_key()

# Serialize group's public key for distribution to verifiers
group_public_key_pem = group_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# Function to generate a member's key pair
def generate_member_key_pair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
    )
    public_key = private_key.public_key()
    return private_key, public_key

# Example: Generate keys for three members
member1_private_key, member1_public_key = generate_member_key_pair()
member2_private_key, member2_public_key = generate_member_key_pair()
member3_private_key, member3_public_key = generate_member_key_pair()

# Function for member to sign a message
def sign_message(private_key, message):
    signature = private_key.sign(
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return signature

# Function for verifier to verify a signature
def verify_signature(public_key, message, signature):
    try:
        public_key.verify(
            signature,
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception as e:
        return False

# Example: Member 1 signs a message
message = b"This is a secret message"
signature = sign_message(member1_private_key, message)

# Example: Verifier verifies the signature
is_valid = verify_signature(member1_public_key, message, signature)
print(f"Signature valid: {is_valid}")


Signature valid: True


In [2]:
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes

# Generate group's master key pair
group_private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)
group_public_key = group_private_key.public_key()

# Serialize group's public key for distribution to verifiers
group_public_key_pem = group_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# Function to generate a member's key pair with age verification
def generate_member_key_pair(age):
    if age < 18:
        raise ValueError("Member must be at least 18 years old for age verification.")
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
    )
    public_key = private_key.public_key()
    return private_key, public_key, age

# Function for member to sign their age
def sign_age(private_key, age):
    age_message = bytes(str(age), 'utf-8')
    signature = private_key.sign(
        age_message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return signature

# Function for verifier to verify a member's age
def verify_age(public_key, age, signature):
    age_message = bytes(str(age), 'utf-8')
    try:
        public_key.verify(
            signature,
            age_message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception as e:
        return False

# Example usage
if __name__ == "__main__":
    # Generate keys for a member with age verification
    try:
        member_private_key, member_public_key, member_age = generate_member_key_pair(20)  # Assume member is 20 years old
        member_signature = sign_age(member_private_key, member_age)

        # Verifier verifies the member's age
        is_valid_age = verify_age(member_public_key, member_age, member_signature)
        print(f"Age verification successful: {is_valid_age}")
    except ValueError as e:
        print(e)


Age verification successful: True
